# Chapter 2: Spectral Graph Theory and Graph Signal Processing

Building on the graph fundamentals from Chapter 1, this notebook dives deep into the mathematical foundations that enable Graph Neural Networks to work effectively. We'll explore spectral graph theory, eigendecomposition, and graph signal processing.

## Learning Objectives
By the end of this notebook, you will understand:
1. Eigenvalues and eigenvectors of graph matrices
2. Graph Laplacian properties and spectral decomposition
3. Graph Fourier Transform and frequency analysis
4. Graph filtering and smoothing operations
5. How spectral methods enable graph convolutions
6. Normalized Laplacian and its applications

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from scipy import linalg
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. The Graph Laplacian Matrix

The **Graph Laplacian** is the cornerstone of spectral graph theory. It encodes the structure of the graph and enables us to analyze graph properties through linear algebra.

### 1.1 Definition and Properties

For a graph G = (V, E) with adjacency matrix A and degree matrix D:

**Combinatorial Laplacian**: L = D - A

**Key Properties**:
- L is symmetric and positive semidefinite
- Smallest eigenvalue is always 0
- Row sums are 0: L·𝟏 = 0 (where 𝟏 is the all-ones vector)
- Number of zero eigenvalues = number of connected components

**Quadratic Form**: For any vector x:
x^T L x = ½ Σ_{(i,j)∈E} (x_i - x_j)²

This measures the "smoothness" of signal x on the graph!

In [ ]:
def analyze_graph_laplacian(G, title="Graph"):
    """Comprehensive analysis of graph Laplacian properties"""
    
    # Get matrices
    A = nx.adjacency_matrix(G).todense()
    degrees = [G.degree(node) for node in G.nodes()]
    D = np.diag(degrees)
    L = D - A
    
    # Eigendecomposition
    eigenvals, eigenvecs = linalg.eigh(L)
    
    # Sort by eigenvalue
    idx = np.argsort(eigenvals)
    eigenvals = eigenvals[idx]
    eigenvecs = eigenvecs[:, idx]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Original Graph
    ax = axes[0, 0]
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, ax=ax, with_labels=True, node_color='lightblue', 
            node_size=1000, font_size=10, font_weight='bold')
    ax.set_title(f'{title}\n{G.number_of_nodes()} nodes, {G.number_of_edges()} edges')
    
    # 2. Laplacian Matrix
    ax = axes[0, 1]
    im = ax.imshow(L, cmap='RdBu', vmin=-np.max(np.abs(L)), vmax=np.max(np.abs(L)))
    for i in range(L.shape[0]):
        for j in range(L.shape[1]):
            ax.text(j, i, f'{L[i,j]:.0f}', ha='center', va='center', 
                    fontsize=8, fontweight='bold', 
                    color='white' if abs(L[i,j]) > np.max(np.abs(L))/2 else 'black')
    ax.set_title('Graph Laplacian Matrix L')
    plt.colorbar(im, ax=ax)
    
    # 3. Eigenvalue Spectrum
    ax = axes[0, 2]
    ax.bar(range(len(eigenvals)), eigenvals, color='skyblue', alpha=0.7)
    ax.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    ax.set_xlabel('Eigenvalue Index')
    ax.set_ylabel('Eigenvalue')
    ax.set_title('Laplacian Eigenspectrum')
    ax.grid(True, alpha=0.3)
    
    # Add eigenvalue annotations
    for i, val in enumerate(eigenvals[:3]):
        ax.annotate(f'λ_{i} = {val:.3f}', (i, val), 
                   textcoords="offset points", xytext=(0,10), ha='center')
    
    # 4. First few eigenvectors visualized on graph
    for idx, (i, eigenval) in enumerate([(0, eigenvals[0]), (1, eigenvals[1]), (2, eigenvals[2])]):
        if idx >= 3:
            break
            
        ax = axes[1, idx]
        
        # Color nodes by eigenvector values
        node_colors = eigenvecs[:, i]
        
        # Create colormap
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_colors, 
                                      cmap='RdBu', node_size=800, 
                                      vmin=-np.max(np.abs(node_colors)), 
                                      vmax=np.max(np.abs(node_colors)))
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.5)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=8, font_weight='bold')
        
        # Add colorbar
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        
        ax.set_title(f'Eigenvector {i+1}\nλ_{i+1} = {eigenval:.3f}')
        ax.axis('off')
        
        # Add eigenvector values as text
        ax.text(0.02, 0.98, f'v_{i+1} = [{"  ".join([f"{val:.2f}" for val in eigenvecs[:, i]])}]', 
                transform=ax.transAxes, fontsize=8, verticalalignment='top',
                bbox=dict(boxstyle="round,pad=0.3", facecolor="lightyellow", alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # Print analysis
    print(f"📊 Spectral Analysis of {title}")
    print(f"{'='*50}")
    print(f"Graph Properties:")
    print(f"  • Nodes: {G.number_of_nodes()}")
    print(f"  • Edges: {G.number_of_edges()}")
    print(f"  • Connected components: {nx.number_connected_components(G)}")
    print(f"  • Is connected: {nx.is_connected(G)}")
    
    print(f"\nLaplacian Properties:")
    print(f"  • Matrix shape: {L.shape}")
    print(f"  • Is symmetric: {np.allclose(L, L.T)}")
    print(f"  • Row sums: {np.sum(L, axis=1).flatten()}")
    print(f"  • Trace (sum of eigenvalues): {np.trace(L):.3f}")
    
    print(f"\nEigenvalue Analysis:")
    print(f"  • Number of zero eigenvalues: {np.sum(np.abs(eigenvals) < 1e-10)}")
    print(f"  • Smallest eigenvalue: {eigenvals[0]:.6f}")
    print(f"  • Second smallest (Fiedler value): {eigenvals[1]:.6f}")
    print(f"  • Largest eigenvalue: {eigenvals[-1]:.3f}")
    print(f"  • Spectral gap: {eigenvals[1] - eigenvals[0]:.6f}")
    
    return L, eigenvals, eigenvecs, pos

# Analyze different graph types
print("🔍 Spectral Analysis of Different Graphs")
print("=======================================\n")

# 1. Path graph
G_path = nx.path_graph(6)
L_path, evals_path, evecs_path, pos_path = analyze_graph_laplacian(G_path, "Path Graph")

In [ ]:
# 2. Cycle graph
G_cycle = nx.cycle_graph(6)
L_cycle, evals_cycle, evecs_cycle, pos_cycle = analyze_graph_laplacian(G_cycle, "Cycle Graph")

In [ ]:
# 3. Complete graph
G_complete = nx.complete_graph(5)
L_complete, evals_complete, evecs_complete, pos_complete = analyze_graph_laplacian(G_complete, "Complete Graph")

## 2. Normalized Graph Laplacian

The **normalized Laplacian** addresses issues with degree heterogeneity and provides better numerical properties for many applications.

### 2.1 Symmetric Normalized Laplacian

**Definition**: L_sym = D^(-1/2) L D^(-1/2) = I - D^(-1/2) A D^(-1/2)

**Properties**:
- Eigenvalues in [0, 2]
- Better conditioning than combinatorial Laplacian
- Preserves symmetry

### 2.2 Random Walk Normalized Laplacian

**Definition**: L_rw = D^(-1) L = I - D^(-1) A

**Properties**:
- Not symmetric but same eigenvalues as L_sym
- Related to random walks on graphs
- Eigenvalues in [0, 2]

In [ ]:
def compare_laplacian_types(G, title="Graph"):
    """Compare different types of graph Laplacians"""
    
    # Get basic matrices
    A = nx.adjacency_matrix(G).todense()
    degrees = np.array([G.degree(node) for node in G.nodes()])
    D = np.diag(degrees)
    
    # Combinatorial Laplacian
    L = D - A
    
    # Symmetric normalized Laplacian
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees + 1e-10))  # Add small epsilon for numerical stability
    L_sym = D_inv_sqrt @ L @ D_inv_sqrt
    
    # Random walk normalized Laplacian  
    D_inv = np.diag(1.0 / (degrees + 1e-10))
    L_rw = D_inv @ L
    
    # Compute eigenvalues
    evals_L = linalg.eigvals(L)
    evals_L_sym = linalg.eigvals(L_sym)
    evals_L_rw = linalg.eigvals(L_rw)
    
    # Sort eigenvalues
    evals_L = np.sort(np.real(evals_L))
    evals_L_sym = np.sort(np.real(evals_L_sym))
    evals_L_rw = np.sort(np.real(evals_L_rw))
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Visualize matrices
    matrices = [(L, "Combinatorial L"), (L_sym, "Symmetric Normalized L_sym"), (L_rw, "Random Walk L_rw")]
    eigenvals = [evals_L, evals_L_sym, evals_L_rw]
    
    for i, ((matrix, name), evals) in enumerate(zip(matrices, eigenvals)):
        # Matrix visualization
        ax = axes[0, i]
        im = ax.imshow(matrix, cmap='RdBu', vmin=-np.max(np.abs(matrix)), vmax=np.max(np.abs(matrix)))
        ax.set_title(f'{name}\n{matrix.shape[0]}×{matrix.shape[1]}')
        plt.colorbar(im, ax=ax, shrink=0.8)
        
        # Add matrix values for small matrices
        if matrix.shape[0] <= 6:
            for row in range(matrix.shape[0]):
                for col in range(matrix.shape[1]):
                    ax.text(col, row, f'{matrix[row,col]:.2f}', ha='center', va='center', 
                            fontsize=8, fontweight='bold',
                            color='white' if abs(matrix[row,col]) > np.max(np.abs(matrix))/2 else 'black')
        
        # Eigenvalue spectrum
        ax = axes[1, i]
        ax.bar(range(len(evals)), evals, alpha=0.7, 
               color=['red', 'orange', 'blue'][i])
        ax.set_xlabel('Eigenvalue Index')
        ax.set_ylabel('Eigenvalue')
        ax.set_title(f'Eigenspectrum of {name}\nRange: [{evals[0]:.3f}, {evals[-1]:.3f}]')
        ax.grid(True, alpha=0.3)
        
        # Add some eigenvalue annotations
        for j, val in enumerate(evals[:min(3, len(evals))]):
            ax.annotate(f'{val:.3f}', (j, val), 
                       textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Numerical comparison
    print(f"📊 Laplacian Comparison for {title}")
    print(f"{'='*60}")
    print(f"{'Property':<25} {'Combinatorial':<15} {'Symmetric':<15} {'Random Walk':<15}")
    print(f"{'-'*60}")
    print(f"{'Min eigenvalue':<25} {evals_L[0]:<15.6f} {evals_L_sym[0]:<15.6f} {evals_L_rw[0]:<15.6f}")
    print(f"{'Max eigenvalue':<25} {evals_L[-1]:<15.3f} {evals_L_sym[-1]:<15.3f} {evals_L_rw[-1]:<15.3f}")
    print(f"{'Condition number':<25} {evals_L[-1]/max(evals_L[1], 1e-10):<15.1f} {evals_L_sym[-1]/max(evals_L_sym[1], 1e-10):<15.1f} {evals_L_rw[-1]/max(evals_L_rw[1], 1e-10):<15.1f}")
    print(f"{'Is symmetric':<25} {np.allclose(L, L.T)!s:<15} {np.allclose(L_sym, L_sym.T)!s:<15} {np.allclose(L_rw, L_rw.T)!s:<15}")
    
    return L, L_sym, L_rw, evals_L, evals_L_sym, evals_L_rw

# Compare Laplacian types on different graphs
print("⚖️ Comparing Different Laplacian Types")
print("====================================\n")

# Test on a small irregular graph
G_test = nx.Graph()
G_test.add_edges_from([(0,1), (1,2), (2,3), (1,3), (3,4), (4,5)])

results = compare_laplacian_types(G_test, "Irregular Graph")

## 3. Graph Fourier Transform

The **Graph Fourier Transform (GFT)** extends classical Fourier analysis to graphs, enabling frequency analysis of graph signals.

### 3.1 Graph Signals
A **graph signal** is a function f: V → ℝ that assigns a value to each vertex.

Examples:
- Temperature at weather stations
- User features in social networks
- Gene expression levels

### 3.2 Graph Fourier Transform Definition

Given the eigendecomposition of the normalized Laplacian L = UΛU^T:

**Forward GFT**: f̂ = U^T f
**Inverse GFT**: f = U f̂

Where:
- U = [u₁, u₂, ..., uₙ] are the eigenvectors (graph frequencies)
- Λ = diag(λ₁, λ₂, ..., λₙ) are the eigenvalues (frequency magnitudes)
- f̂ are the Fourier coefficients

### 3.3 Frequency Interpretation
- **Low frequencies** (small eigenvalues): Smooth signals across the graph
- **High frequencies** (large eigenvalues): Rapidly changing signals

In [ ]:
def demonstrate_graph_fourier_transform(G, signals=None):
    """Demonstrate Graph Fourier Transform on different signals"""
    
    # Get normalized Laplacian
    A = nx.adjacency_matrix(G).todense()
    degrees = np.array([G.degree(node) for node in G.nodes()])
    D = np.diag(degrees)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees + 1e-10))
    L_sym = D_inv_sqrt @ (D - A) @ D_inv_sqrt
    
    # Eigendecomposition (Graph Fourier basis)
    eigenvals, eigenvecs = linalg.eigh(L_sym)
    idx = np.argsort(eigenvals)
    eigenvals = eigenvals[idx]
    eigenvecs = eigenvecs[:, idx]
    
    # Create example signals if none provided
    if signals is None:
        n_nodes = G.number_of_nodes()
        signals = {
            'Constant': np.ones(n_nodes),
            'Linear': np.arange(n_nodes, dtype=float),
            'Random': np.random.randn(n_nodes),
            'Localized': np.zeros(n_nodes)
        }
        signals['Localized'][n_nodes//2] = 1.0  # Impulse at center
    
    pos = nx.spring_layout(G, seed=42)
    
    fig, axes = plt.subplots(len(signals), 4, figsize=(20, 5*len(signals)))
    if len(signals) == 1:
        axes = axes.reshape(1, -1)
    
    for i, (signal_name, signal) in enumerate(signals.items()):
        # Normalize signal for better visualization
        signal_norm = (signal - np.mean(signal)) / (np.std(signal) + 1e-10)
        
        # 1. Original signal on graph
        ax = axes[i, 0]
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=signal_norm, 
                                      cmap='RdBu', node_size=600,
                                      vmin=-2, vmax=2)
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=8)
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        ax.set_title(f'{signal_name} Signal\nf = {signal}')
        ax.axis('off')
        
        # 2. Graph Fourier Transform
        gft_coeffs = eigenvecs.T @ signal
        
        ax = axes[i, 1]
        bars = ax.bar(range(len(gft_coeffs)), np.abs(gft_coeffs), 
                     color=['red' if abs(c) > 0.1 else 'lightblue' for c in gft_coeffs],
                     alpha=0.7)
        ax.set_xlabel('Frequency Index (eigenvalue order)')
        ax.set_ylabel('|Fourier Coefficient|')
        ax.set_title(f'GFT Magnitude Spectrum\n|f̂| = |U^T f|')
        ax.grid(True, alpha=0.3)
        
        # Add eigenvalue annotations
        ax2 = ax.twinx()
        ax2.plot(range(len(eigenvals)), eigenvals, 'ro-', alpha=0.5, markersize=4)
        ax2.set_ylabel('Eigenvalue (frequency)', color='red')
        ax2.tick_params(axis='y', labelcolor='red')
        
        # 3. Low-pass filtered signal (keep only low frequencies)
        n_low_freq = max(1, len(eigenvals) // 3)
        gft_coeffs_lowpass = gft_coeffs.copy()
        gft_coeffs_lowpass[n_low_freq:] = 0
        signal_lowpass = eigenvecs @ gft_coeffs_lowpass
        
        ax = axes[i, 2]
        signal_lowpass_norm = (signal_lowpass - np.mean(signal_lowpass)) / (np.std(signal_lowpass) + 1e-10)
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=signal_lowpass_norm, 
                                      cmap='RdBu', node_size=600,
                                      vmin=-2, vmax=2)
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=8)
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        ax.set_title(f'Low-pass Filtered\n(keep {n_low_freq} lowest frequencies)')
        ax.axis('off'
        
        # 4. High-pass filtered signal
        gft_coeffs_highpass = gft_coeffs.copy()
        gft_coeffs_highpass[:n_low_freq] = 0
        signal_highpass = eigenvecs @ gft_coeffs_highpass
        
        ax = axes[i, 3]
        signal_highpass_norm = (signal_highpass - np.mean(signal_highpass)) / (np.std(signal_highpass) + 1e-10)
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=signal_highpass_norm, 
                                      cmap='RdBu', node_size=600,
                                      vmin=-2, vmax=2)
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=8)
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        ax.set_title(f'High-pass Filtered\n(remove {n_low_freq} lowest frequencies)')
        ax.axis('off')
        
        # Print analysis
        print(f"\n📈 GFT Analysis: {signal_name} Signal")
        print(f"Original signal energy: {np.sum(signal**2):.3f}")
        print(f"GFT coefficients energy: {np.sum(gft_coeffs**2):.3f}")
        print(f"Energy preservation (Parseval): {np.allclose(np.sum(signal**2), np.sum(gft_coeffs**2))}")
        print(f"Dominant frequencies: {np.argsort(np.abs(gft_coeffs))[-3:][::-1]}")
        print(f"Low-pass reconstruction error: {np.linalg.norm(signal - signal_lowpass):.3f}")
    
    plt.tight_layout()
    plt.show()
    
    return eigenvals, eigenvecs, gft_coeffs

print("🌊 Graph Fourier Transform Demonstration")
print("=======================================\n")

# Create a graph for demonstration
G_demo = nx.grid_2d_graph(3, 3)  # 3x3 grid
G_demo = nx.convert_node_labels_to_integers(G_demo)

eigenvals, eigenvecs, gft_coeffs = demonstrate_graph_fourier_transform(G_demo)

## 4. Graph Filtering and Convolution

Graph filtering is the foundation of Graph Neural Networks. It enables us to perform convolution-like operations on irregular graph structures.

### 4.1 Graph Filtering

A **graph filter** is a linear operator that modifies graph signals based on the graph structure.

**Spectral Definition**: H = U h(Λ) U^T
- h(λ) is the filter function
- Applied to eigenvalues: h(Λ) = diag(h(λ₁), h(λ₂), ..., h(λₙ))

**Filtered Signal**: y = H x = U h(Λ) U^T x

### 4.2 Common Graph Filters

1. **Low-pass filter**: h(λ) = exp(-τλ) - emphasizes smooth signals
2. **High-pass filter**: h(λ) = λ - emphasizes oscillatory signals  
3. **Band-pass filter**: Selects specific frequency ranges
4. **Polynomial filters**: h(λ) = Σ aₖλᵏ

### 4.3 Localized Filters

For computational efficiency, we want **localized** filters that only consider K-hop neighborhoods.

**Polynomial filters** achieve this:
H = Σ_{k=0}^K θₖ Lᵏ

This only requires matrix powers of L, avoiding eigendecomposition!

In [ ]:
def demonstrate_graph_filtering(G, signal, filter_types=['lowpass', 'highpass', 'polynomial']):
    """Demonstrate different types of graph filtering"""
    
    # Get normalized Laplacian and its eigendecomposition
    A = nx.adjacency_matrix(G).todense()
    degrees = np.array([G.degree(node) for node in G.nodes()])
    D = np.diag(degrees)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees + 1e-10))
    L_sym = D_inv_sqrt @ (D - A) @ D_inv_sqrt
    
    eigenvals, eigenvecs = linalg.eigh(L_sym)
    idx = np.argsort(eigenvals)
    eigenvals = eigenvals[idx]
    eigenvecs = eigenvecs[:, idx]
    
    pos = nx.spring_layout(G, seed=42)
    
    fig, axes = plt.subplots(2, len(filter_types) + 1, figsize=(5*(len(filter_types)+1), 10))
    
    # Original signal
    ax = axes[0, 0]
    signal_norm = (signal - np.mean(signal)) / (np.std(signal) + 1e-10)
    nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=signal_norm, 
                                  cmap='RdBu', node_size=800, vmin=-2, vmax=2)
    nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
    nx.draw_networkx_labels(G, pos, ax=ax, font_size=10, font_weight='bold')
    plt.colorbar(nodes, ax=ax, shrink=0.8)
    ax.set_title('Original Signal')
    ax.axis('off')
    
    # Filter response functions
    ax = axes[1, 0]
    lambda_range = np.linspace(0, np.max(eigenvals), 100)
    
    filters = {}
    
    for i, filter_type in enumerate(filter_types):
        if filter_type == 'lowpass':
            # Exponential low-pass filter
            tau = 1.0
            h_lambda = np.exp(-tau * eigenvals)
            h_range = np.exp(-tau * lambda_range)
            color = 'blue'
            label = f'Low-pass (τ={tau})'
            
        elif filter_type == 'highpass':
            # Linear high-pass filter
            h_lambda = eigenvals / (np.max(eigenvals) + 1e-10)
            h_range = lambda_range / (np.max(eigenvals) + 1e-10)
            color = 'red'
            label = 'High-pass (linear)'
            
        elif filter_type == 'polynomial':
            # Polynomial filter (Chebyshev approximation of low-pass)
            # Simple 2nd order polynomial
            theta = [0.5, -0.3, 0.1]  # Coefficients
            h_lambda = theta[0] + theta[1] * eigenvals + theta[2] * eigenvals**2
            h_range = theta[0] + theta[1] * lambda_range + theta[2] * lambda_range**2
            color = 'green'
            label = f'Polynomial (order {len(theta)-1})'
        
        filters[filter_type] = h_lambda
        
        # Plot filter response
        ax.plot(lambda_range, h_range, color=color, linewidth=2, label=label)
        ax.scatter(eigenvals, h_lambda, color=color, s=50, zorder=5)
    
    ax.set_xlabel('Eigenvalue λ')
    ax.set_ylabel('Filter Response h(λ)')
    ax.set_title('Filter Response Functions')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Apply filters and visualize results
    for i, filter_type in enumerate(filter_types):
        h_lambda = filters[filter_type]
        
        # Apply filter: y = U h(Λ) U^T x
        filtered_signal = eigenvecs @ (h_lambda[:, np.newaxis] * (eigenvecs.T @ signal[:, np.newaxis]))
        filtered_signal = filtered_signal.flatten()
        
        # Visualize filtered signal
        ax = axes[0, i+1]
        filtered_norm = (filtered_signal - np.mean(filtered_signal)) / (np.std(filtered_signal) + 1e-10)
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=filtered_norm, 
                                      cmap='RdBu', node_size=800, vmin=-2, vmax=2)
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=10, font_weight='bold')
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        ax.set_title(f'{filter_type.title()} Filtered')
        ax.axis('off')
        
        # Frequency domain analysis
        ax = axes[1, i+1]
        gft_original = eigenvecs.T @ signal
        gft_filtered = eigenvecs.T @ filtered_signal
        
        x_pos = np.arange(len(eigenvals))
        width = 0.35
        
        ax.bar(x_pos - width/2, np.abs(gft_original), width, alpha=0.7, 
               label='Original', color='gray')
        ax.bar(x_pos + width/2, np.abs(gft_filtered), width, alpha=0.7, 
               label='Filtered', color=['blue', 'red', 'green'][i])
        
        ax.set_xlabel('Frequency Index')
        ax.set_ylabel('|GFT Coefficient|')
        ax.set_title(f'{filter_type.title()} - Frequency Domain')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Print analysis
        print(f"\n🔍 {filter_type.title()} Filter Analysis:")
        print(f"  Signal energy before: {np.sum(signal**2):.3f}")
        print(f"  Signal energy after: {np.sum(filtered_signal**2):.3f}")
        print(f"  Energy ratio: {np.sum(filtered_signal**2)/np.sum(signal**2):.3f}")
        print(f"  Filter values: {h_lambda}")
    
    plt.tight_layout()
    plt.show()
    
    return filters

print("🎛️ Graph Filtering Demonstration")
print("==============================\n")

# Create test signal (noisy signal with trend)
G_filter = nx.path_graph(8)
np.random.seed(42)
test_signal = np.array([0, 1, 3, 2, 4, 3, 5, 4]) + 0.5 * np.random.randn(8)

filters = demonstrate_graph_filtering(G_filter, test_signal)

## 5. Connection to Graph Neural Networks

Now we can see how spectral graph theory enables Graph Neural Networks!

### 5.1 Spectral Graph Convolution

**Graph convolution** can be defined as filtering with learnable parameters:

y = gθ ★ x = U gθ(Λ) U^T x

Where gθ(λ) is a learnable filter function parameterized by θ.

### 5.2 Polynomial Approximation

To avoid expensive eigendecomposition, we use **polynomial approximations**:

gθ(L) = Σ_{k=0}^K θₖ Lᵏ

This gives us **localized convolutions** with computational complexity O(|E|)!

### 5.3 ChebNet and GCN

- **ChebNet**: Uses Chebyshev polynomials for efficient approximation
- **GCN**: Simplifies to first-order approximation with renormalization trick

### 5.4 From Spectral to Spatial

This spectral foundation leads to spatial interpretations:
- Convolution = Aggregating information from neighbors
- Filter order K = Size of receptive field
- Learnable weights = Feature transformations

In [ ]:
def demonstrate_polynomial_filters(G, signal, max_order=4):
    """Demonstrate polynomial filters and their localization properties"""
    
    # Get normalized Laplacian
    A = nx.adjacency_matrix(G).todense()
    degrees = np.array([G.degree(node) for node in G.nodes()])
    D = np.diag(degrees)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(degrees + 1e-10))
    L_sym = D_inv_sqrt @ (D - A) @ D_inv_sqrt
    
    pos = nx.spring_layout(G, seed=42)
    
    fig, axes = plt.subplots(2, max_order + 1, figsize=(4*(max_order+1), 8))
    
    # Original signal
    ax = axes[0, 0]
    signal_norm = (signal - np.mean(signal)) / (np.std(signal) + 1e-10)
    nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=signal_norm, 
                                  cmap='RdBu', node_size=800, vmin=-2, vmax=2)
    nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
    nx.draw_networkx_labels(G, pos, ax=ax, font_size=10, font_weight='bold')
    plt.colorbar(nodes, ax=ax, shrink=0.8)
    ax.set_title('Original Signal')
    ax.axis('off')
    
    # Show receptive fields
    center_node = len(signal) // 2
    ax = axes[1, 0]
    node_colors = ['red' if i == center_node else 'lightblue' for i in range(len(signal))]
    nx.draw(G, pos, ax=ax, node_color=node_colors, node_size=800, with_labels=True, 
            font_size=10, font_weight='bold')
    ax.set_title(f'Center Node: {center_node}')
    ax.axis('off')
    
    # Apply polynomial filters of different orders
    L_power = np.eye(len(signal))  # L^0 = I
    
    for k in range(1, max_order + 1):
        L_power = L_power @ L_sym  # L^k
        
        # Simple polynomial filter: just L^k
        filtered_signal = L_power @ signal
        
        # Visualize filtered signal
        ax = axes[0, k]
        filtered_norm = (filtered_signal - np.mean(filtered_signal)) / (np.std(filtered_signal) + 1e-10)
        nodes = nx.draw_networkx_nodes(G, pos, ax=ax, node_color=filtered_norm, 
                                      cmap='RdBu', node_size=800, vmin=-2, vmax=2)
        nx.draw_networkx_edges(G, pos, ax=ax, alpha=0.3)
        nx.draw_networkx_labels(G, pos, ax=ax, font_size=10, font_weight='bold')
        plt.colorbar(nodes, ax=ax, shrink=0.8)
        ax.set_title(f'L^{k} Filter\n(Order {k})')
        ax.axis('off')
        
        # Show k-hop neighborhood of center node
        ax = axes[1, k]
        
        # Get k-hop neighbors
        k_hop_neighbors = set([center_node])
        current_neighbors = set([center_node])
        
        for hop in range(k):
            next_neighbors = set()
            for node in current_neighbors:
                next_neighbors.update(G.neighbors(node))
            current_neighbors = next_neighbors - k_hop_neighbors
            k_hop_neighbors.update(current_neighbors)
        
        # Color nodes by hop distance
        node_colors = []
        for i in range(len(signal)):
            if i == center_node:
                node_colors.append('red')
            elif i in k_hop_neighbors:
                # Get actual hop distance
                try:
                    hop_dist = nx.shortest_path_length(G, center_node, i)
                    if hop_dist <= k:
                        intensity = 1.0 - (hop_dist / k) * 0.7
                        node_colors.append((1.0, intensity, intensity))
                    else:
                        node_colors.append('lightgray')
                except:
                    node_colors.append('lightgray')
            else:
                node_colors.append('lightgray')
        
        nx.draw(G, pos, ax=ax, node_color=node_colors, node_size=800, with_labels=True, 
                font_size=10, font_weight='bold')
        ax.set_title(f'{k}-hop Neighborhood\n(Receptive Field)')
        ax.axis('off')
        
        print(f"\n📍 Order {k} Analysis:")
        print(f"  Receptive field size: {len(k_hop_neighbors)} nodes")
        print(f"  Signal propagation: {k} hops from center")
        print(f"  Filter effect: {np.std(filtered_signal):.3f} (higher = more variation)")
    
    plt.tight_layout()
    plt.show()

print("🎯 Polynomial Filters and Localization")
print("====================================\n")

# Create a path graph to clearly see localization
G_path = nx.path_graph(9)
impulse_signal = np.zeros(9)
impulse_signal[4] = 1.0  # Impulse at center

demonstrate_polynomial_filters(G_path, impulse_signal, max_order=4)

## 6. Summary and Key Takeaways

This chapter introduced the fundamental mathematical tools that make Graph Neural Networks possible.

### 🎯 Key Concepts Mastered

1. **Graph Laplacian**: L = D - A encodes graph structure
2. **Spectral Decomposition**: L = UΛU^T provides frequency analysis
3. **Graph Fourier Transform**: Analyzes signals in frequency domain
4. **Graph Filtering**: Modifies signals based on graph structure
5. **Polynomial Filters**: Enable localized, efficient operations

### 🔗 Connection to GNNs

The spectral tools we learned directly enable:
- **Graph Convolution**: y = gθ ★ x = U gθ(Λ) U^T x
- **Localization**: Polynomial filters limit to K-hop neighborhoods
- **Efficiency**: Avoid eigendecomposition using polynomial approximation
- **Learning**: Parameters θ are learned from data

### 📚 Mathematical Foundations

**Core Equations to Remember**:
- Quadratic form: x^T L x = ½ Σ_{(i,j)∈E} (x_i - x_j)²
- GFT: f̂ = U^T f, f = U f̂
- Filtering: y = U h(Λ) U^T x
- Polynomial: H = Σ_{k=0}^K θₖ Lᵏ

### 🚀 What's Next

In Chapter 3, we'll see how these spectral concepts translate into practical message passing frameworks that form the backbone of modern Graph Neural Networks!

In [ ]:
# Create chapter summary
def create_spectral_summary():
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Spectral decomposition concept
    ax = axes[0, 0]
    # Create a simple graph to show eigenvalue/eigenvector concept
    G_simple = nx.path_graph(5)
    A = nx.adjacency_matrix(G_simple).todense()
    D = np.diag([G_simple.degree(node) for node in G_simple.nodes()])
    L = D - A
    
    eigenvals, eigenvecs = linalg.eigh(L)
    
    ax.bar(range(len(eigenvals)), eigenvals, color='skyblue', alpha=0.7)
    ax.set_title('Spectral Decomposition\nL = UΛU^T', fontsize=14, fontweight='bold')
    ax.set_xlabel('Eigenvalue Index')
    ax.set_ylabel('Eigenvalue (Frequency)')
    ax.grid(True, alpha=0.3)
    
    # 2. Graph filtering concept
    ax = axes[0, 1]
    lambda_range = np.linspace(0, 2, 100)
    lowpass = np.exp(-lambda_range)
    highpass = lambda_range / 2
    polynomial = 0.5 - 0.3 * lambda_range + 0.1 * lambda_range**2
    
    ax.plot(lambda_range, lowpass, 'b-', linewidth=2, label='Low-pass')
    ax.plot(lambda_range, highpass, 'r-', linewidth=2, label='High-pass')
    ax.plot(lambda_range, polynomial, 'g-', linewidth=2, label='Polynomial')
    
    ax.set_title('Graph Filter Functions\nh(λ)', fontsize=14, fontweight='bold')
    ax.set_xlabel('Eigenvalue λ')
    ax.set_ylabel('Filter Response h(λ)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 3. Localization through polynomial orders
    ax = axes[1, 0]
    orders = [0, 1, 2, 3, 4]
    receptive_field_sizes = [1, 2, 3, 4, 5]  # For path graph
    
    bars = ax.bar(orders, receptive_field_sizes, color='orange', alpha=0.7)
    ax.set_title('Localization Property\nReceptive Field Growth', fontsize=14, fontweight='bold')
    ax.set_xlabel('Polynomial Order K')
    ax.set_ylabel('Receptive Field Size')
    ax.grid(True, alpha=0.3)
    
    # Add annotations
    for i, (order, size) in enumerate(zip(orders, receptive_field_sizes)):
        ax.annotate(f'{size} nodes', (order, size), 
                   textcoords="offset points", xytext=(0,5), ha='center')
    
    # 4. From spectral to spatial
    ax = axes[1, 1]
    ax.text(0.5, 0.9, 'Spectral → Spatial Translation', ha='center', 
            fontsize=16, fontweight='bold', transform=ax.transAxes)
    
    spectral_spatial = [
        ('Eigenvalues λ', 'Graph frequencies'),
        ('Eigenvectors U', 'Spatial patterns'),
        ('Filter h(λ)', 'Feature transformation'),
        ('Polynomial order K', 'Neighborhood size'),
        ('y = UhU^T x', 'Message passing'),
        ('Learnable θ', 'Neural network weights')
    ]
    
    for i, (spectral, spatial) in enumerate(spectral_spatial):
        y_pos = 0.75 - i * 0.1
        ax.text(0.1, y_pos, spectral, fontsize=12, fontweight='bold', 
                transform=ax.transAxes, color='blue')
        ax.text(0.5, y_pos, '→', fontsize=14, ha='center', 
                transform=ax.transAxes)
        ax.text(0.55, y_pos, spatial, fontsize=12, 
                transform=ax.transAxes, color='red')
    
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Learning progress
    print("\n📚 Learning Progress:")
    print("✅ Chapter 1: Graph Fundamentals")
    print("✅ Chapter 2: Spectral Graph Theory")
    print("⬜ Chapter 3: Message Passing")
    print("⬜ Chapter 4: Graph Convolutions")
    print("⬜ Chapter 5: Advanced GNNs")
    print("⬜ Chapter 6: Applications")

print("🎓 Chapter 2 Summary: Spectral Graph Theory")
print("==========================================\n")

create_spectral_summary()

print("\n🏆 Congratulations! You've mastered spectral graph theory!")
print("\nYou now understand:")
print("• How graph structure encodes in the Laplacian matrix")
print("• Eigendecomposition and frequency analysis of graphs")
print("• Graph Fourier Transform and signal processing")
print("• Graph filtering and convolution operations")
print("• Polynomial approximations for computational efficiency")
print("• The mathematical foundation underlying all GNNs")

print("\n🔜 Next: Chapter 3 - Message Passing Framework")
print("Ready to see how spectral theory translates to practical algorithms!")